Based on [Tensorflow tutorial](https://www.tensorflow.org/tutorials/text/word2vec?hl=en#prepare_training_data_for_word2vec).

To adapt w2v to book recomendation the books replace the words. List of rated books by an user is equivalent to a "sentence". The book vectorization is already made by the database.

Current issues :

*   try to fit the book data in this implementation ==> only keep a fix amount of book for each other (10 books)
  * need to make more lists for each user in order to have all the book represented??
Since every list is independent, need to make book appears several time for different list of each user with different books.
  * Increase the size of each list to fit the longest user list and fill the smaller ones with 0?
* book selection includes every rating of user. Need to remove books from bad ratings.
*   Not sure what the *vocab_size* is supposed to be, the number of unique books in the data set is slightly too small.
* Only use one dataset, will switch to other 


---
# Books
---


## W2V

In [ ]:
import io
import itertools # This module implements a number of iterator building blocks inspired by constructs from APL, Haskell (brrr), and SML
import os
import re #regex
import string

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import tqdm # make loops show a smart progress meter: 76%|████████████████████████       | 7568/10000 [00:33<00:10, 229.00it/s]
            # cute
import tqdm.notebook as tq # better for notebook


import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

AUTOTUNE = tf.data.AUTOTUNE # need to read about tf Pipeline, and Prefetching
SEED = 42 
num_ns = 4

**num_ns**: negative sampled context words that do not appear in the window size neighborhood of target_word

A skip-gram model predicts the context (or neighbors) of a word, given the word itself. 
The training objective of the skip-gram model is to maximize the probability of predicting context words given the target word

The Noise Contrastive Estimation loss function is an efficient approximation for a full softmax. With an objective to learn word embeddings instead of modelling the word distribution, NCE loss can be simplified to use negative sampling.

The simplified negative sampling objective for a target word is to distinguish the context word from num_ns negative samples drawn from noise distribution of words. More precisely, an efficient approximation of full softmax over the vocabulary is, for a skip-gram pair, to pose the loss for a target word as a classification problem between the context word and num_ns negative samples.



In [ ]:

rated = pd.read_csv("https://raw.githubusercontent.com/ArielNATAF/Books_project/dev/datasets/ratings.csv", usecols = ["user_id","book_id"])
rated = rated.sort_values('user_id')

rated["book_id"].nunique()
books = pd.read_csv("https://raw.githubusercontent.com/ArielNATAF/Books_project/dev/datasets/books.csv", usecols =['book_id', 'original_title'])
rated.head()


In [ ]:
# Split lists of ds in list of 10:
#   • Select a list in the ds (a list: a row in book_list column)
#   • Check size lists of the ds
#   • If not multiple of 10 itemss in list
#       • Pad with 0 to multiple of 10 items
#   • If more than 10 items in list
#       • Copy last 10 items of list in new list
#       • Remove last 10 items of current list
#       • Add the new list to the ds
#   • Go to next list

# Need to be vastly optimized.
#   • Create a  temp Series with number of cell to add,
#   • Add the lists in each cells pf temp
#   • append the filled temp after the loop
# (should divide by 10 the number of appends)

# Currently only a small part of dataset to save the RAM
import timeit

def split_list(ds):

  def pad_list(lst):
    if not lst.size % 10 == 0:
      lst = np.pad(lst, (0, 10 - (lst.size % 10)), mode='constant', constant_values=0)
    return lst

  print('count at start:',ds.count())
  # count at start: 53424
  start = timeit.default_timer()

  print(ds[0].size,ds[1].size,ds[2].size,ds[3].size,ds[4].size)
  ds = ds.apply(lambda x: pad_list(x))
  print(ds[0].size,ds[1].size,ds[2].size,ds[3].size,ds[4].size)

  for i in tq.tqdm(range(3000)):
	  while ds[i].size > 10:
	    size = ds[i].size
	    indice=[size-10,size-9,size-8,size-7,size-6,size-5,size-4,size-3,size-2,size-1]
	    new_list=ds[i][indice]
	    ds[i] = np.delete(ds[i],indice,0)
	    ds = ds.append(pd.Series([new_list]), ignore_index=True)

  print('count at end:',ds.count())
  # count at end: 621633
    
  stop = timeit.default_timer()
  print('Time: ', int(stop - start)) # Time:  3200 => 53mn
  return ds

In [ ]:
#Group the books of each user in lists in one column
user_ratedbooks_ds = rated.groupby('user_id')['book_id'].apply(list).reset_index(name='book_list')
user_ratedbooks_ds = user_ratedbooks_ds.drop('user_id', 1)
user_ratedbooks_ds['book_list'].to_numpy()

user_ratedbooks_ds['book_list'] = user_ratedbooks_ds['book_list'].apply(lambda x: np.array(x))
user_ratedbooks_ds['book_list'] = split_list(user_ratedbooks_ds['book_list'])
# Too big, will eat most of RAM memory, so only affect first few thousands of row
user_ratedbooks_ds['book_list'] = user_ratedbooks_ds['book_list'].apply(lambda x: x[[0,1,2,3,4,5,6,7,8,9]])
# Bad way to only have 10 book for each list. Cut the "excess"
  # ==> Need to improve making of lists
# No user has less than 10 books rated (min=19) currently.
  # function to check min/max sizes in ##Misc section

print(user_ratedbooks_ds.head())
print(type(user_ratedbooks_ds['book_list'][0]))

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tq.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=SEED, 
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
# I am not sure what the vocab size is supposed to be,
# the number of unique book is slightly too small here
vocab_size = int(rated["book_id"].nunique()*1.2) 
vocab      = books["original_title"].tolist()
type(books["original_title"].tolist())

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences   = user_ratedbooks_ds['book_list'], 
    window_size = 2, 
    num_ns      = 4, 
    vocab_size  = vocab_size, 
    seed        = SEED)
print(len(targets), len(contexts), len(labels))

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)


In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)


In [ ]:
def custom_loss(x_logit, y_true):
   return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)


embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=15, callbacks=[tensorboard_callback])


In [ ]:
%load_ext tensorboard

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

%tensorboard --logdir logs

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = books["original_title"].tolist()

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if  index == 0: continue # skip 0, it's padding.
  vec = weights[index] 
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(str(word) + "\n")
out_v.close()
out_m.close()

<

< `vectors.tsv` & `metadata.tsv` in the notebook repo

< 


In [ ]:
#download the files
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass

## Misc.

In [ ]:
def check_min(ds=user_ratedbooks_ds['book_list']):
  ds = ds.apply(lambda x: x.size)
  print("Min size:", ds.min())
  print("Max size:", ds.max())

check_min()

---
# TextExample

---



In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [ ]:
with open(path_to_file) as f: 
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [ ]:
# We create a custom standardization function to lowercase the text and 
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

# Define the vocabulary size and number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))
type(vectorize_layer)

In [ ]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])


In [ ]:
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))

# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
text_vector_ds

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")
  print(type (seq))


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=SEED, 
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences, 
    window_size=2, 
    num_ns=4, 
    vocab_size=vocab_size, 
    seed=SEED)
print(len(targets), len(contexts), len(labels))


In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)


In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)


In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
embedding_dim = 128

word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])


In [ ]:
%load_ext tensorboard

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

%tensorboard --logdir logs
